<a href="https://colab.research.google.com/github/Ashok401/AIML_BootCamp/blob/main/Capstone/Capstone_ABSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Aspect-based Sentiment Analysis

- Explore the possibility of replacing expensive and slow LLMs with smaller, custom models. These models should be simple and scalable, while still maintaining acceptable quality.
- Scaling the model to work on datasets containing 500 and 1000 reviews, which are currently in the top 100 reviews.
- Observation: By employing Knowledge Distillation, we leverage an LLM (Teacher) to train a set of Logistic Regression “Expert” models (Students). These models attain an impressive 85%+ agreement with the LLM, all while incurring a 99.9% lower cost.

In [ ]:
#Data set : https://www.kaggle.com/datasets/mrmars1010/iphone-customer-reviews-nlp

import pandas as pd
import json
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from openai import OpenAI
from google.colab import userdata

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.multioutput import MultiOutputClassifier


**iPhone Aspect-Based Sentiment Analysis (ABSA)**:
Distilling LLM Intelligence into High-Performance Classical ML

By using Knowledge Distillation, we utilize an LLM (Teacher) to train a suite of Logistic Regression "Expert" models (Students) that achieve 85%+ agreement with the LLM at 99.9% lower cost.

🚀 **The Problem & Solution**
The Problem: LLMs (like GPT-4 or Gemini) are highly accurate but too slow and expensive for real-time analysis of millions of iPhone 14/15/16 reviews.

The Solution: A "Hybrid" approach. We use the LLM to label a "Gold Standard" dataset of 1,000 reviews, then train a local, optimized TF-IDF + Logistic Regression pipeline to replicate that logic.

📊 **Key Achievements**
Aspect	Accuracy (vs LLM)	Status

Camera	94%	Elite Performance

Battery	86%	Production Ready

Performance	84%	Production Ready

Display	79%	Stable Baseline

Performance Gain: Inference latency reduced from ~1s/review (LLM) to <1ms/review (Local).


🛠️ **Technical Implementation**
1. **Aspect Modeling**

To ensure the model focuses on technical features, we use a structured aspect_map to filter reviews into specific categories:

Battery: ['battery', 'charge', 'drain', 'backup', 'power']

Camera: ['photo', 'video', 'lens', 'zoom', 'selfie', 'night mode']

Display: ['screen', 'oled', 'brightness', 'refresh', 'hz', 'pixel']

Performance: ['fast', 'lag', 'speed', 'processor', 'gaming', 'hang']

2. **Model Optimization**

The "Student" models were optimized using:

Bigram Analysis (ngram_range=(1,2)): To understand negations like "not fast."

Class Balancing: Applied class_weight='balanced' to ensure the model catches negative complaints despite a positive data bias.

Feature Capping: Limited to 1,000 features to prevent overfitting on specific iPhone 14/15 terminology, ensuring compatibility with future models (iPhone 16+).

In [ ]:
# Use the top 500 reviews  for aspect-based sentiment analysis.
# Train the model with LLM labels and compare it to classical machine learning models to assess its performance compared to LLMs.
# Conduct the experiment again with the top 1000 reviews to observe the impact on the model’s performance.

datasets = [
    {
        "name": "top500_reviews",
        "raw": "top500_reviews.csv",
        "labeled": "top500_reviews_with_llm_labels.csv",
        "max_features": 500
    },
    {
        "name": "top1000_reviews",
        "raw": "top1000_reviews.csv",
        "labeled": "top1000_reviews_with_llm_labels.csv",
        "max_features": 1000
    }
]

OPEN_AI_KEY = userdata.get('OPEN_AI_KEY')

client = OpenAI(
    api_key = OPEN_AI_KEY
)

aspect_map = {
    'llm_Battery': ['battery', 'charge', 'drain', 'backup', 'power'],
    'llm_Display': ['screen', 'oled', 'brightness', 'refresh', 'hz', 'pixel'],
    'llm_Camera': ['photo', 'video', 'lens', 'zoom', 'selfie', 'night mode'],
    'llm_Performance': ['fast', 'lag', 'speed', 'processor', 'gaming', 'hang']
}

def get_llm_labels(review_text):
    # Identify which aspects are actually in the text
    text_lower = review_text.lower()
    found_aspects = [a for a, keywords in aspect_map.items() if any(k in text_lower for k in keywords)]

    if not found_aspects:
        return {"General": 1 if "good" in text_lower else 0}

    # Construct the prompt for ONLY the found aspects
    prompt = f"""
    Analyze the sentiment for the following aspects in this phone review: {found_aspects}
    Return ONLY a JSON object where 1 is positive and  0 is negative.
        Review: "{review_text}"
    Example Output: {{"llm_Battery": 1, "llm_Camera": 0}}
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        response_format={ "type": "json_object" },
        temperature=0
    )
    return json.loads(response.choices[0].message.content)

for ds in datasets:
  print(f"Metrics for '{ds['name']}':\n")

  csv_file = ds["labeled"] if os.path.exists(ds["labeled"]) else ds["raw"]
  df = pd.read_csv(csv_file)

  llm_labels = list(aspect_map.keys())
  llm_already_labled =  set(llm_labels).issubset(df.columns)

  if llm_already_labled:
    print("LLM labels are already available. Therefore, skipping the call to an LLM.\n")
  else:
    labels = df['review'].apply(lambda x: get_llm_labels(x))
    labels_df = pd.json_normalize(labels)

    df = pd.concat([df.reset_index(drop=True), labels_df.reset_index(drop=True)], axis=1)
    df.to_csv(ds["labeled"], index=False)

  for col in llm_labels:
    mask = df[col].notna()
    X = df.loc[mask, 'review']
    y = df.loc[mask, col].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = Pipeline([
        ('tfidf', TfidfVectorizer(stop_words = 'english', max_features=ds["max_features"], ngram_range=(1, 2))),
        ('clf', LogisticRegression(solver='liblinear', class_weight='balanced'))
    ])

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\nMetrics for '{col}':\n")
    print(f"classification_report:\n")
    print(classification_report(y_test, y_pred))
    print(f"accuracy_score:")
    print(accuracy_score(y_test, y_pred))
    print(f"\nconfusion_matrix:")
    print(confusion_matrix(y_test, y_pred))




Metrics for 'top500_reviews':

LLM labels are already available. Therefore, skipping the call to an LLM.


Metrics for 'llm_Battery':

classification_report:

              precision    recall  f1-score   support

         0.0       0.79      0.94      0.86        16
         1.0       0.97      0.90      0.94        40

    accuracy                           0.91        56
   macro avg       0.88      0.92      0.90        56
weighted avg       0.92      0.91      0.91        56

accuracy_score:
0.9107142857142857

confusion_matrix:
[[15  1]
 [ 4 36]]

Metrics for 'llm_Display':

classification_report:

              precision    recall  f1-score   support

         0.0       0.65      0.85      0.73        13
         1.0       0.88      0.70      0.78        20

    accuracy                           0.76        33
   macro avg       0.76      0.77      0.76        33
weighted avg       0.79      0.76      0.76        33

accuracy_score:
0.7575757575757576

confusion_matrix:
[[11  2